# 원, 삼각형, 사각형 구분하기

In [1]:
import numpy as np
import tensorflow as tf
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)


## 데이터셋 생성하기

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/225.)
train_generator = train_datagen.flow_from_directory( # 디렉터리로 가져옴 0 / 데이터프레임x
    'handwriting_shape/train',
    target_size=(24,24),
    batch_size=3,
    class_mode= 'categorical'
) 

Found 45 images belonging to 3 classes.


In [10]:
test_datagen = ImageDataGenerator(rescale=1/225.)
test_generator = test_datagen.flow_from_directory(
    'handwriting_shape/test',
    target_size=(24,24),
    batch_size=3,
    class_mode= 'categorical'
)

Found 15 images belonging to 3 classes.


In [8]:
train_generator.labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2])

In [9]:
train_generator.filenames[0]

'circle\\circle001.png'

## 모델 정의 / 설정 / 학습

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense 

In [14]:
model = Sequential([
    Conv2D(32, (3,3), input_shape= (24,24,3) ,activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation= 'relu'),
    Dense(3, activation= 'softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 22, 22, 32)        896       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               819328    
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 839,107
Trainable params: 839,107
Non-trainable params: 0
________________________________________________

In [16]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## 모델 학습

- 첫번째 인자 : 훈련데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 train_generator으로 지정.
- steps_per_epoch : 한 epoch에 사용한 스텝 수를 지정. 총 45개의 훈련 샘플이 있고 배치사이즈가 3이므로 15 스텝으로 지정.
- epochs : 전체 훈련 데이터셋에 대해 학습 반복 횟수를 지정. 50번을 반복적으로 학습.
- validation_data : 검증데이터셋을 제공할 제네레이터를 지정. 본 예제에서는 앞서 생성한 test_generator으로 지정.
- validation_steps : 한 epoch 종료 시 마다 검증할 때 사용되는 검증 스텝 수를 지정. 총 15개의 검증 샘플이 있고 배치사이즈가 3이므로 5 스텝으로 지정

In [17]:
 model.fit_generator(  # fix x fit_generator o 
    train_generator,
    steps_per_epoch=15,
    epochs=50,
    validation_data = test_generator,
    validation_steps=5,
 )

Epoch 1/50
15/15 [==============================] - 0s 32ms/step - loss: 1.5360 - accuracy: 0.4889 - val_loss: 0.9420 - val_accuracy: 0.6667
Epoch 2/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6150 - accuracy: 0.7556 - val_loss: 0.3563 - val_accuracy: 1.0000
Epoch 3/50
15/15 [==============================] - 0s 9ms/step - loss: 0.1470 - accuracy: 1.0000 - val_loss: 0.0900 - val_accuracy: 0.9333
Epoch 4/50
15/15 [==============================] - 0s 10ms/step - loss: 0.0189 - accuracy: 1.0000 - val_loss: 0.0457 - val_accuracy: 1.0000
Epoch 5/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0299 - val_accuracy: 1.0000
Epoch 6/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.0399 - val_accuracy: 1.0000
Epoch 7/50
15/15 [==============================] - 0s 9ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.0365 - val_accuracy: 1.0000
Epoch 8/50
15/15 [

## 모델 평가

In [18]:
model.evaluate_generator(test_generator, steps =5)
# warning massage -> 이제 generator 쓸 필요 x

Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.01236023847013712, 1.0]

In [19]:
#  generator 지워줌
model.evaluate(test_generator, steps =5)

5/5 [==============================] - 0s 3ms/step - loss: 0.0124 - accuracy: 1.0000


[0.01236023847013712, 1.0]